In [1]:
import pandas as pd

In [7]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.8 kB/s eta 0:00:0000:0100:04m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 134.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 80.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 KB 71.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 KB 98.5 kB/s eta 0:00:0000:0100:01


In [2]:
data = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/final_entries_v2.tsv', sep='\t')
data

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families
0,1a0n,a,B,PPRPLPVAPGSSKT,P06241,MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTS...,"Protein kinase superfamily, Tyr protein kinase..."
1,1a1m,c,A,TPYDINQML,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
2,1a1n,c,A,VPLRPMTY,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
3,1a1o,c,A,KPIVQYDNF,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot
4,1a6a,c,A,PVSKMRMATPLLMQA,P01903,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,MHC class II family
...,...,...,...,...,...,...,...
3465,DB11995,NaN,NaN,APSNLAS,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3466,DB11995,NaN,NaN,QQWSFNPPT,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3467,DB11995,NaN,NaN,GYTFTSYNMH,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."
3468,DB11995,NaN,NaN,AIYPGNGDTSYNQKFKG,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ..."


In [3]:
unique_peptides = data['pep_seq'].nunique()
unique_proteins = data['prot_seq'].nunique()

print(f"Jumlah peptida unik: {unique_peptides}")
print(f"Jumlah protein unik: {unique_proteins}")


Jumlah peptida unik: 2771
Jumlah protein unik: 895


### ProtT5 

In [4]:
!pip install sentencepiece

import torch
import gc
import pickle
import pandas as pd
import os
import sys
from transformers import T5Tokenizer, T5EncoderModel

data = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/final_entries_v2.tsv', sep='\t')

unique_peptides = data['pep_seq'].drop_duplicates().tolist()
unique_proteins = data['prot_seq'].drop_duplicates().tolist()

print('Memuat model dan tokenizer ProtT5...')

tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_bfd")
model.eval()

def get_prott5_embedding(sequence: str, len_seq_limit: int):
    sequence_w_spaces = ' '.join(list(sequence))
    encoded_input = tokenizer(
        sequence_w_spaces,
        max_length=len_seq_limit,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    with torch.no_grad():
        output = model(**encoded_input)
    output_hidden = output.last_hidden_state
    mean_embedding = output_hidden.mean(dim=1).squeeze().cpu().numpy()
    return mean_embedding

def save_checkpoint(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_checkpoint(path):
    try:
        with open(path, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return []

# Lokasi file
checkpoint_prot_path = '/home/g6523023/ProtT5 ekstraksi fitur/checkpoint_protein_.pkl'
checkpoint_pep_path = '/home/g6523023/ProtT5 ekstraksi fitur/checkpoint_peptida_.pkl'

embed_vects_list_prot = load_checkpoint(checkpoint_prot_path)
start_idx_prot = len(embed_vects_list_prot)

for idx, entry in enumerate(range(start_idx_prot, len(unique_proteins))):
    print('Memproses urutan protein unik: ', idx + start_idx_prot)
    embed_vects_list_prot.append(
        get_prott5_embedding(sequence=unique_proteins[entry], len_seq_limit=1500)
    )
    if (idx + 1) % 20 == 0:
        save_checkpoint(embed_vects_list_prot, checkpoint_prot_path)
        print(f'Checkpoint protein unik disimpan pada indeks: {entry}')
    gc.collect()

save_checkpoint(embed_vects_list_prot, checkpoint_prot_path)
print('Checkpoint protein terakhir disimpan.')

prot_df = pd.DataFrame(embed_vects_list_prot, columns=[f'prot_embed_{i}' for i in range(1024)])
prot_df.to_csv('/home/g6523023/ProtT5 ekstraksi fitur/protein_embeddings.csv', index=False)
print('Hasil ekstraksi protein disimpan dalam format CSV.')

embed_vects_list_pep = load_checkpoint(checkpoint_pep_path)
start_idx_pep = len(embed_vects_list_pep)

for idx, entry in enumerate(range(start_idx_pep, len(unique_peptides))):
    print('Memproses urutan peptida unik: ', idx + start_idx_pep)
    embed_vects_list_pep.append(
        get_prott5_embedding(sequence=unique_peptides[entry], len_seq_limit=50)
    )
    if (idx + 1) % 20 == 0:
        save_checkpoint(embed_vects_list_pep, checkpoint_pep_path)
        print(f'Checkpoint peptida unik disimpan pada indeks: {entry}')
    gc.collect()

save_checkpoint(embed_vects_list_pep, checkpoint_pep_path)
print('Checkpoint peptida terakhir disimpan.')

pep_df = pd.DataFrame(embed_vects_list_pep, columns=[f'pep_embed_{i}' for i in range(1024)])
pep_df.to_csv('/home/g6523023/ProtT5 ekstraksi fitur/peptide_embeddings.csv', index=False)
print('Hasil ekstraksi peptida disimpan dalam format CSV.')

Memuat model dan tokenizer ProtT5...


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Memproses urutan protein unik:  500
Memproses urutan protein unik:  501
Memproses urutan protein unik:  502
Memproses urutan protein unik:  503
Memproses urutan protein unik:  504
Memproses urutan protein unik:  505
Memproses urutan protein unik:  506
Memproses urutan protein unik:  507
Memproses urutan protein unik:  508
Memproses urutan protein unik:  509
Memproses urutan protein unik:  510
Memproses urutan protein unik:  511
Memproses urutan protein unik:  512
Memproses urutan protein unik:  513
Memproses urutan protein unik:  514
Memproses urutan protein unik:  515
Memproses urutan protein unik:  516
Memproses urutan protein unik:  517
Memproses urutan protein unik:  518
Memproses urutan protein unik:  519
Checkpoint protein unik disimpan pada indeks: 519
Memproses urutan protein unik:  520
Memproses urutan protein unik:  521
Memproses urutan protein unik:  522
Memproses urutan protein unik:  523
Memproses urutan protein unik:  524
Memproses urutan protein unik:  525
Memproses urut

In [6]:
pep_embeddings = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/peptide_embeddings.csv')
prot_embeddings = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/protein_embeddings.csv')
data = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/final_entries_v2.tsv', sep='\t')

In [7]:
pep_embeddings

,pep_embed_0,pep_embed_1,pep_embed_2,pep_embed_3,pep_embed_4,pep_embed_5,pep_embed_6,pep_embed_7,pep_embed_8,pep_embed_9,...,pep_embed_1014,pep_embed_1015,pep_embed_1016,pep_embed_1017,pep_embed_1018,pep_embed_1019,pep_embed_1020,pep_embed_1021,pep_embed_1022,pep_embed_1023
0,0.030033,-0.100224,0.079272,0.227686,-0.079435,0.039185,-0.292841,-0.230378,-0.196702,-0.083397,...,-0.063308,0.027649,0.173081,-0.091212,-0.020554,-0.151844,-0.037160,0.105167,-0.027322,0.085864
1,0.038422,-0.160526,0.090410,0.389460,-0.191433,-0.107539,-0.167799,-0.287230,-0.119073,-0.325324,...,-0.141910,-0.194485,0.110324,-0.038524,0.114788,0.027494,-0.086945,0.144176,-0.171401,0.037446
2,-0.086734,-0.043309,0.225117,0.375819,-0.259559,-0.067419,-0.255602,-0.368272,-0.146283,-0.341365,...,-0.256578,-0.070564,0.210806,-0.175142,0.113861,-0.029000,-0.128377,0.201383,-0.069374,0.003012
3,-0.092968,-0.132457,0.107277,0.409582,-0.231267,-0.073200,-0.321279,-0.368311,-0.152260,-0.199685,...,-0.249954,-0.172648,0.147120,-0.182624,0.089883,0.065010,-0.081457,0.216630,-0.218383,-0.027238
4,0.156240,-0.147064,0.019275,0.213881,-0.166329,0.024147,-0.118274,-0.221780,-0.190452,-0.272740,...,-0.118667,-0.037750,0.224005,-0.104574,0.099576,-0.071385,-0.095890,0.008189,-0.157015,0.079378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,0.175287,-0.131529,0.086486,0.187616,-0.073843,-0.011382,-0.173533,-0.189571,-0.271536,-0.091747,...,-0.037079,-0.113313,0.187927,-0.013537,-0.010985,-0.268374,-0.080741,0.118247,-0.080962,0.036411
2767,0.260573,-0.034742,0.175675,0.211602,0.112913,0.046684,-0.127765,-0.339423,-0.225960,-0.271697,...,-0.100499,-0.235268,0.117621,-0.093461,0.113751,-0.325200,0.077547,0.127065,-0.108274,-0.009086
2768,0.025862,-0.099464,0.167782,0.390458,-0.211102,-0.103873,-0.136846,-0.294010,-0.155570,-0.340221,...,-0.252320,-0.157768,0.124186,-0.054363,0.127524,0.048685,-0.232772,0.164395,-0.205319,0.088387
2769,0.090063,-0.116827,0.112314,0.209173,-0.221868,-0.030083,-0.127324,-0.174748,-0.139345,-0.233667,...,-0.131384,-0.126336,0.103540,-0.123216,0.103988,-0.069987,-0.086469,-0.001757,-0.053871,-0.071445


In [8]:
prot_embeddings

,prot_embed_0,prot_embed_1,prot_embed_2,prot_embed_3,prot_embed_4,prot_embed_5,prot_embed_6,prot_embed_7,prot_embed_8,prot_embed_9,...,prot_embed_1014,prot_embed_1015,prot_embed_1016,prot_embed_1017,prot_embed_1018,prot_embed_1019,prot_embed_1020,prot_embed_1021,prot_embed_1022,prot_embed_1023
0,-0.083235,0.055176,0.071488,-0.088650,-0.031568,-0.052288,-0.140325,-0.137256,0.023072,0.144468,...,-0.083720,0.027851,-0.063560,-0.056856,0.227259,-0.024167,-0.018593,0.004605,-0.111024,-0.023716
1,0.095172,0.039099,-0.006942,-0.127286,0.062951,-0.032198,-0.216347,-0.047011,0.143662,0.354119,...,-0.194320,0.281502,0.077305,0.006572,0.128127,-0.117371,-0.196142,-0.134186,0.034336,0.244518
2,-0.049262,-0.122468,-0.046796,-0.041865,0.086802,0.020302,-0.052127,-0.229399,0.116252,0.213562,...,0.011935,0.071232,0.088962,-0.083966,0.262632,-0.018609,-0.064899,0.034401,-0.062924,0.120452
3,-0.021687,-0.041057,-0.032828,-0.095424,0.069276,0.004880,-0.094773,-0.215641,0.108164,0.238657,...,-0.096916,0.114093,0.092764,-0.043886,0.241528,0.017557,-0.103925,0.027826,-0.032020,0.193608
4,0.092903,0.035765,-0.004715,-0.104932,0.073599,-0.057860,-0.212637,-0.023273,0.134905,0.343422,...,-0.172625,0.266271,0.072947,-0.007819,0.154349,-0.152263,-0.173195,-0.120776,0.011140,0.249772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,-0.055203,-0.098685,0.023796,-0.063312,0.024434,-0.072433,-0.217213,-0.104724,0.010392,0.017330,...,-0.047477,0.001733,0.025266,-0.083113,0.180174,-0.010245,0.007328,0.024472,0.016580,0.076871
891,-0.011266,-0.012389,0.075903,-0.038897,0.013400,0.036004,-0.109413,-0.121055,0.065317,0.099260,...,-0.029067,0.060793,0.014047,-0.125572,0.086528,-0.032146,0.034582,0.025381,0.058212,0.036418
892,-0.026814,-0.122247,-0.026273,-0.044251,-0.021572,0.082705,-0.149719,-0.120598,0.008041,0.063425,...,0.075430,0.037689,0.059854,-0.048890,0.053570,-0.009694,-0.053496,-0.139912,-0.061757,0.100482
893,0.082780,0.025162,0.153158,0.203322,-0.073528,-0.143167,-0.069562,-0.218389,0.203641,0.008150,...,-0.148647,0.025582,0.057740,0.016342,0.280953,-0.239537,-0.117746,0.044918,-0.262422,-0.038987


In [9]:
unique_peptides = data['pep_seq'].drop_duplicates().tolist()
unique_proteins = data['prot_seq'].drop_duplicates().tolist()

In [10]:
pep_embed_dict = {seq: embedding for seq, embedding in zip(unique_peptides, pep_embeddings.values)}
prot_embed_dict = {seq: embedding for seq, embedding in zip(unique_proteins, prot_embeddings.values)}

data['pep_embeddings'] = data['pep_seq'].map(pep_embed_dict)
data['prot_embeddings'] = data['prot_seq'].map(prot_embed_dict)

In [11]:
pep_emb_df = pd.DataFrame(data['pep_embeddings'].tolist(), columns=[f'pep_embed_{i}' for i in range(1024)])
prot_emb_df = pd.DataFrame(data['prot_embeddings'].tolist(), columns=[f'prot_embed_{i}' for i in range(1024)])

final_df = pd.concat([data, pep_emb_df, prot_emb_df], axis=1)

In [12]:
final_df.to_csv('/home/g6523023/ProtT5 ekstraksi fitur/ProtT5_1tot1_new.csv', index=False)
print('Dataset akhir dengan embedding berhasil disimpan!')

Dataset akhir dengan embedding berhasil disimpan!


In [13]:
protT5 = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/ProtT5_1tot1_new.csv')

/tmp/ipykernel_431552/1994219268.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  protT5 = pd.read_csv('/home/g6523023/ProtT5 ekstraksi fitur/ProtT5_1tot1_new.csv')


In [14]:
protT5

,pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families,pep_embeddings,prot_embeddings,pep_embed_0,...,prot_embed_1014,prot_embed_1015,prot_embed_1016,prot_embed_1017,prot_embed_1018,prot_embed_1019,prot_embed_1020,prot_embed_1021,prot_embed_1022,prot_embed_1023
0,1a0n,a,B,PPRPLPVAPGSSKT,P06241,MGCVQCKDKEATKLTEERDGSLNQSSGYRYGTDPTPQHYPSFGVTS...,"Protein kinase superfamily, Tyr protein kinase...",[ 0.03003258 -0.10022359 0.07927243 ... 0.10...,[-0.08323541 0.05517614 0.0714876 ... 0.00...,0.030033,...,-0.083720,0.027851,-0.063560,-0.056856,0.227259,-0.024167,-0.018593,0.004605,-0.111024,-0.023716
1,1a1m,c,A,TPYDINQML,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot,[ 0.03842222 -0.16052558 0.09040973 ... 0.14...,[ 0.09517173 0.03909942 -0.00694231 ... -0.13...,0.038422,...,-0.194320,0.281502,0.077305,0.006572,0.128127,-0.117371,-0.196142,-0.134186,0.034336,0.244518
2,1a1n,c,A,VPLRPMTY,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot,[-0.08673408 -0.04330897 0.22511692 ... 0.20...,[ 0.09517173 0.03909942 -0.00694231 ... -0.13...,-0.086734,...,-0.194320,0.281502,0.077305,0.006572,0.128127,-0.117371,-0.196142,-0.134186,0.034336,0.244518
3,1a1o,c,A,KPIVQYDNF,P01889,MLVMAPRTVLLLLSAALALTETWAGSHSMRYFYTSVSRPGRGEPRF...,Unknown_from_uniprot,[-0.0929681 -0.13245735 0.10727675 ... 0.21...,[ 0.09517173 0.03909942 -0.00694231 ... -0.13...,-0.092968,...,-0.194320,0.281502,0.077305,0.006572,0.128127,-0.117371,-0.196142,-0.134186,0.034336,0.244518
4,1a6a,c,A,PVSKMRMATPLLMQA,P01903,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,MHC class II family,[ 0.1562404 -0.14706391 0.0192747 ... 0.00...,[-0.04926218 -0.1224676 -0.04679602 ... 0.03...,0.156240,...,0.011935,0.071232,0.088962,-0.083966,0.262632,-0.018609,-0.064899,0.034401,-0.062924,0.120452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,DB11995,NaN,NaN,APSNLAS,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ...",[ 0.17528671 -0.13152924 0.08648562 ... 0.11...,[-0.07288244 0.05593102 0.16818243 ... 0.19...,0.175287,...,-0.140761,-0.122004,0.006616,-0.065082,0.167971,-0.198251,0.001589,0.190163,-0.146684,-0.080644
3466,DB11995,NaN,NaN,QQWSFNPPT,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ...",[ 0.26057336 -0.03474166 0.17567463 ... 0.12...,[-0.07288244 0.05593102 0.16818243 ... 0.19...,0.260573,...,-0.140761,-0.122004,0.006616,-0.065082,0.167971,-0.198251,0.001589,0.190163,-0.146684,-0.080644
3467,DB11995,NaN,NaN,GYTFTSYNMH,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ...",[ 0.02586176 -0.09946445 0.16778171 ... 0.16...,[-0.07288244 0.05593102 0.16818243 ... 0.19...,0.025862,...,-0.140761,-0.122004,0.006616,-0.065082,0.167971,-0.198251,0.001589,0.190163,-0.146684,-0.080644
3468,DB11995,NaN,NaN,AIYPGNGDTSYNQKFKG,Q9UNQ0,MSSSNVEVFIPVSQGNTNGFPATASNDLKAFTEGAVLSFHNICYRV...,"ABC transporter superfamily, ABCG family, Eye ...",[ 0.09006339 -0.11682692 0.11231418 ... -0.00...,[-0.07288244 0.05593102 0.16818243 ... 0.19...,0.090063,...,-0.140761,-0.122004,0.006616,-0.065082,0.167971,-0.198251,0.001589,0.190163,-0.146684,-0.080644
